# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the dataset

In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [ ]:
# view first 2 rows of movies
movies.head(2)

In [ ]:
# view first 2 rows of credits
credits.head(2)

In [ ]:
# check how many rows and columns avalable in movies dataset
movies.shape

In [ ]:
# check how many rows and columns avalable in credits dataset
credits.shape

# Merge Datasets

In [ ]:
movies = movies.merge(credits, on='title')

In [ ]:
# view first 2 rows of new movies dataset
movies.head(2)

In [ ]:
# check how many rows and columns avalable in new movies dataset
movies.shape

# Preprocess the data

In [ ]:
# check what are the clomns in new movies dataset
movies.columns

In [ ]:
# choose only some needed colomns only
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'vote_count']]

In [ ]:
# print new movie dataset
movies

In [ ]:
# check no of colomns and rows
movies.shape

Remove missing values

In [ ]:
# check missing values
movies.isnull().sum()

In [ ]:
# drop missing values
movies.dropna(inplace=True)

In [ ]:
# check again missing values in here
movies.isnull().sum()

In [ ]:
# check no of colomns and rows (without missing values)
movies.shape

Remove duplicate values

In [ ]:
# check duplicated movies in here
movies.duplicated().sum()

Covert Genres

In [ ]:
# get first genres
movies.iloc[0]['genres']

In [ ]:
# check type of genres
type(movies.iloc[0]['genres'])

In [ ]:
import ast # it can convert string to list

# create convert function
def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
        
    return l


In [ ]:
# apply the function of genres
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
# view first 2 rows of movies
movies.head(2)

Covert Keywords

In [ ]:
# get first keywords
movies.iloc[0]['keywords']

In [ ]:
# apply the function of keywords
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
# view first 2 rows of movies
movies.head(2)

Convert Cast

In [ ]:
# get first cast
movies.iloc[0]['cast']

In [ ]:
# create convert_cast function
def convert_cast(text):
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            l.append(i['name'])
        counter+=1
    return l

In [ ]:
# apply the function of cast
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
# view first 2 rows of movies
movies.head(2)

Covert Crew

In [ ]:
# get first crew
movies.iloc[0]['crew']

In [ ]:
# create fetch_directory function
def fetch_directory(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
        
    return l

In [ ]:
# apply the function of crew
movies['crew'] = movies['crew'].apply(fetch_directory)

In [ ]:
# view first 2 rows of movies
movies.head(2)

Data spliting

In [ ]:
# get first overview
movies.iloc[0]['overview']

In [ ]:
#split data from the 'overview' column
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
# view first 2 rows of movies
movies.head(2)

In [ ]:
# get first overview
movies.iloc[0]['overview']

In [ ]:
# view first few rows
movies.head()

Remove Spaces 

In [ ]:
# create remove_space function
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(" ",""))
    return l

In [ ]:
# call remove_space function
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
# view first few rows
movies.head()

In [ ]:
#Concatenate the columns & create a new column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
# view first few rows
movies.head()

In [ ]:
movies.iloc[0]['tags']

In [ ]:
# create new data frame
new_df = movies[['movie_id', 'title', 'tags', 'vote_count']]

In [ ]:
# view fist few rows of new data frame
new_df.head()

In [ ]:
# converts tage coloumn to the string format
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(map(str, x)) if isinstance(x, list) else str(x))

In [ ]:
# view first few rows
new_df.head()

In [ ]:
# access the tags colomn in the new_df data frame
new_df.iloc[0]['tags']

In [ ]:
# convert the tags colomn to the lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
# view first few rows
new_df.head()

In [ ]:
# again access the tags colomn in the new_df data frame
new_df.iloc[0]['tags']

In [ ]:
# veiw first few rows of new_df
new_df.head()

# Generating Embeddings

In [ ]:
# import libraries 
import nltk
from nltk.stem import PorterStemmer

In [ ]:
# create an instance of the Porter Stemmer(tool of stemming words)
ps = PorterStemmer()

In [ ]:
# create stem function
def stem(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
        
    return " ".join(l)

In [ ]:
# applying stemming function
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
# access the tags colomn in the new_df data frame
new_df.iloc[0]['tags']

In [ ]:
# call CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')  # create instanse 

In [ ]:
# convert 'tags' into a numerical vector
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
# call the vector
vector

In [ ]:
# representing the number of rows and columns in the array
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# calculate the cosine similarity between vectors
similary = cosine_similarity(vector)

In [ ]:
similary

In [ ]:
# returns a tuple of (number of rows, number of columns)
similary.shape

In [ ]:
# retrieve the index of the row
new_df[new_df['title'] == 'Spider-Man'].index[0]